反序列化OpenQASM文件对象，并运行量子线路。

In [12]:
"""Deserialize OpenQASM file object."""
import os
from qutrunk.circuit import QCircuit

circuit = QCircuit.load(file=os.getcwd() + "/qutrunk/example/bell_pair.qasm", format="openqasm")
# run circuit
res = circuit.run(shots=100)
# print result
print(res.get_counts())

[{"00": 54}, {"11": 46}]


以OpenQASM格式导出量子线路

In [5]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, CNOT, Measure

# allocate
qc = QCircuit()
qr = qc.allocate(2)

# apply gate
H * qr[0]
CNOT * (qr[0], qr[1])
Measure * qr[0]
Measure * qr[1]

# export
qc.dump(file="bell_pair.qasm", format="openqasm")

with open(file="bell_pair.qasm") as f:
    for line in f:
        print(line, end="")

OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q[0] -> c[0];
measure q[1] -> c[1];


反序列化QuSL文件对象，并运行量子线路。

In [13]:
"""Deserialize QuSL file object."""
import os
from qutrunk.circuit import QCircuit

circuit = QCircuit.load(file=os.getcwd() + "/qutrunk/example/bell_pair.qusl")
# run circuit
res = circuit.run(shots=100)
# print result
print(res.get_counts())

[{"00": 57}, {"11": 43}]


申请多个量子寄存器

In [6]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, CNOT, Measure

qc = QCircuit()
# allocate 2 qureg: q1, q2
q1, q2 = qc.allocate([1, 1])

H * q1[0]
CNOT * (q1[0], q2[0])
Measure * q1[0]
Measure * q2[0]
res = qc.run(shots=100)
print(res.get_counts())

0
0
[{"00": 46}, {"11": 54}]


获取线路所有状态对应的概率

In [7]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, CNOT

qc = QCircuit()
# allocate 2 qureg: q1, q2
q = qc.allocate(2)

H * q[0]
CNOT * (q[0], q[1])
print(qc.get_probs())

[{'idx': 0, 'prob': 0.4999999999999999}, {'idx': 1, 'prob': 0.0}, {'idx': 2, 'prob': 0.0}, {'idx': 3, 'prob': 0.4999999999999999}]


参数化量子线路

In [8]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import Ry

circuit = QCircuit()
q = circuit.allocate(2)
theta, phi = circuit.create_parameters(["theta", "phi"])

Ry(theta) * q[0]
Ry(phi) * q[1]

c1 = circuit.bind_parameters({"theta": 1, "phi": 1})
print(c1.get_statevector())

c2 = circuit.bind_parameters({"theta": 2, "phi": 2})
print(c2.get_statevector())

[0.77015115+0.j 0.42073549+0.j 0.42073549+0.j 0.22984885+0.j]
[0.29192658+0.j 0.45464871+0.j 0.45464871+0.j 0.70807342+0.j]


追加线路

In [9]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, CNOT, Measure, X, All

circ1 = QCircuit()
qr = circ1.allocate(2)

# apply gate
H * qr[0]
CNOT * (qr[0], qr[1])

circuit = QCircuit()
q = circuit.allocate(2)

circuit.append_circuit(circ1)

All(Measure) * q

# run circuit
circuit.print()
res = circuit.run(shots=100)
print(res.get_counts())

qreg q[2]
creg c[2]
H * q[0]
MCX(1) * (q[0], q[1])
Measure * q[0]
Measure * q[1]
[{"00": 45}, {"11": 55}]


计算线路期望值

In [10]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import Ry, PauliZ, PauliI

circuit = QCircuit()
q = circuit.allocate(2)

Ry(1.23) * q[0]
pauli_str = [PauliZ(q[0]), PauliI(q[1])]
expect = circuit.expval_pauli(pauli_str)
print(expect)

0.33423772712450267


计算线路期望值之和

In [ ]:
from qutrunk.circuit import QCircuit
from qutrunk.circuit.gates import H, Ry, PauliZ, PauliI, PauliX, PauliCoeff, PauliType, PauliCoeffs

circuit = QCircuit()
q = circuit.allocate(2)

H * q[0]
Ry(1.23) * q[1]

pauli_coeffs = PauliCoeffs() << PauliCoeff(0.12, [PauliType.PAULI_Z]) \
    << PauliCoeff(0.34, [PauliType.PAULI_X, PauliType.PAULI_I])

expect_sum = circuit.expval_pauli_sum(pauli_coeffs)
print(expect_sum)